# YOLOP conversion

### Install necessary libraries

In [ ]:
# !pip install onnxruntime-gpu
# !pip install onnx-simplifier

### Load Torch Model and convert to ONNX

In [ ]:
import onnx
import onnxruntime as ort
import onnxsim
import os
from lib.models import get_net
from lib.utils.utils import select_device
from lib.config import cfg
import torch


save_dir = "/home/ubuntu/workspace/ywshin/construct/YOLOP/onnx"
onnx_path = os.path.join(save_dir, "yolopv2.onnx")
yolop_weight = "/home/ubuntu/workspace/ywshin/construct/YOLOP/runs/Patrasche/221014_from_scratch/epoch-400.pth"
inputs = torch.randn(1, 3, 384, 640)

do_simplify = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = get_net(cfg)
checkpoint = torch.load(yolop_weight, map_location= device)
model.load_state_dict(checkpoint['state_dict'])
model.eval()


print(f"Converting to {onnx_path}")
torch.onnx.export(model, inputs, onnx_path,
                  verbose=False, opset_version=12, input_names=['images'],
                  output_names=['det_out', 'drive_area_seg', 'lane_line_seg'])
print('convert', onnx_path, 'to onnx finish!!!')
# Checks
model_onnx = onnx.load(onnx_path)  # load onnx model
onnx.checker.check_model(model_onnx)  # check onnx model
print(onnx.helper.printable_graph(model_onnx.graph))  # print

onnx.save(model_onnx, onnx_path)

if do_simplify:
    print(f'simplifying with onnx-simplifier {onnxsim.__version__}...')
    model_onnx, check = onnxsim.simplify(model_onnx, check_n=3)
    assert check, 'assert check failed'
    

x = inputs.cpu().numpy()
try:
    sess = ort.InferenceSession(onnx_path, providers=["CUDAExecutionProvider", "TensorrtExecutionProvider"])

    for ii in sess.get_inputs():
        print("Input: ", ii)
    for oo in sess.get_outputs():
        print("Output: ", oo)

    print('read onnx using onnxruntime sucess')
except Exception as e:
    print('read failed')
    raise e

### Load Onnx Model

In [ ]:
# Create dummy input
dummy_input = torch.randn(1, 3, 384, 640).numpy()

# Load onnx model using CUDA
ort_sess = ort.InferenceSession('/home/ubuntu/workspace/ywshin/construct/YOLOP/onnx/yolopv2.onnx', providers=['CUDAExecutionProvider'])

### Run Inference

In [ ]:
# %%timeit
outputs = ort_sess.run(None, {'images': dummy_input})

In [ ]:
outputs[4].shape